In [4]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [13]:
# Create json file

def nested_dd(): 
    return defaultdict(nested_dd) 

def default_to_regular(d):
    if isinstance(d, defaultdict):
        d = {k: default_to_regular(v) for k, v in d.items()}
    return d

d = nested_dd()
d['reg']

for X in ['RBG', 'FBG', 'all']:
    for y_type in ['ogtt', 'bg']:
        d['reg'][X][y_type] = {}
    
# with open(r'..\models\dont_overwrite_hyperparameters.json', 'w') as outfile:
#     json.dump(default_to_regular(d), outfile, indent=2)

In [12]:
default_to_regular(d)

{'reg': {'RBG': {'ogtt': {}, 'bg': {}},
  'FBG': {'ogtt': {}, 'bg': {}},
  'all': {'ogtt': {}, 'bg': {}}}}

In [ ]:
def grid_search(X, y, model, params, cv, scoring, print_all=True, **kwargs):
    gs = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring=scoring,
        cv=cv,
        n_jobs=7,
        error_score='raise',
        **kwargs)
    result = gs.fit(X, y)
    print("Best: %f using %s" % (result.best_score_, result.best_params_))
    means = result.cv_results_['mean_test_score']
    stds = result.cv_results_['std_test_score']
    params = result.cv_results_['params']
    if print_all:
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
    l = []
    for mean, param in zip(means, params):
        d = {'mean': mean, **param}
        l.append(d)
    return pd.DataFrame(l)